<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/pdipm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd

In [0]:
#example problem - 2 batches , 2 variable qp
nbatch=2
nBatch=nbatch
nz=2
nineq=2
neq=1
#to do: extract dimensions from problem parameters + check/add batch dimension
Q=torch.tensor([[4,1,1,2],[6,2,2,2]]).view(nbatch,nz,nz).type(torch.DoubleTensor)
p=torch.tensor([[1,1],[1,6]]).view(nbatch,nz).type(torch.DoubleTensor)
G=torch.tensor([[-1,0,0,-1],[-1,0,0,-1]]).view(nbatch,nineq,nz).type(torch.DoubleTensor)
h=torch.tensor([[0,0],[0,0]]).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor([[1,1],[2,3]]).view(nbatch,neq,nz).type(torch.DoubleTensor)
b=torch.tensor([[1],[4]]).view(nbatch,neq).type(torch.DoubleTensor)

In [0]:
#check if Q is psd:
for i in range(nbatch):
  e,_=torch.eig(Q[i])
  if not torch.all(e[:,0]>0):
    raise RuntimeError("Q is not PSD")

In [0]:
def lu_hack(x):
    data, pivots = x.lu(pivot=not x.is_cuda)
    if x.is_cuda:
        if x.ndimension() == 2:
            pivots = torch.arange(1, 1+x.size(0)).int().cuda()
        elif x.ndimension() == 3:
            pivots = torch.arange(
                1, 1+x.size(1),
            ).unsqueeze(0).repeat(x.size(0), 1).int().cuda()
        else:
            assert False
    return (data, pivots)

In [0]:
def bdiag(d):
    nBatch, sz = d.size()
    D = torch.zeros(nBatch, sz, sz).type_as(d)
    I = torch.eye(sz).repeat(nBatch, 1, 1).type_as(d).bool()
    D[I] = d.squeeze().view(-1)
    return D

In [0]:
def get_Hessian(Q,G,A):
    nbatch,nineq,nz=G.size()
    neq=A.size()[1]
    B1=torch.zeros(nbatch,nz+nineq,nz+nineq).type_as(Q)
    B3=torch.zeros(nbatch,neq+nineq,nz+nineq).type_as(Q)
    B4=torch.zeros(nbatch,neq+nineq,neq+nineq).type_as(Q)

    B1[:,:nz,:nz]=Q
    B1[:,-nineq:,-nineq:]=torch.eye(nineq).repeat(nbatch,1,1).type_as(Q)

    B3[:,:nineq,:nz]=G
    B3[:,-neq:,:nz]=A
    B3[:,:nineq,nineq:]=torch.eye(nineq).repeat(nbatch,1,1).type_as(Q)

    B2=torch.transpose(B3, dim0=2, dim1=1)

    H=torch.cat((torch.cat((B1,B2),dim=2),torch.cat((B3,B4),dim=2)),dim=1)
  
    return H

In [0]:
def solve_kkt(H,rx,rs,rz,ry,d=None):
    if d!=None:
      D=bdiag(d)
      H[:,nz:nz+nineq,nz:nz+nineq]=D
    
    F=torch.cat((rx,rs,rz,ry), dim=1).unsqueeze(2)
    H_lu,H_piv= lu_hack(H)
    step=F.lu_solve(H_lu,H_piv)

    rx=step[:,:nz,:]
    rs=step[:,nz:nz+nineq,:]
    rz=step[:,nz+nineq:-neq,:]
    ry=step[:,-neq:,:]
    return(rx,rs,rz,ry)

In [21]:
H=get_Hessian(Q,G,A)
#initial solution
solve_kkt(H,-p,torch.zeros(nbatch,nineq).type_as(Q),h,b)

(tensor([[[0.3333],
          [0.6667]],
 
         [[0.5294],
          [0.9804]]], dtype=torch.float64), tensor([[[0.3333],
          [0.6667]],
 
         [[0.5294],
          [0.9804]]], dtype=torch.float64), tensor([[[-0.3333],
          [-0.6667]],
 
         [[-0.5294],
          [-0.9804]]], dtype=torch.float64), tensor([[[-3.3333]],
 
         [[-3.3333]]], dtype=torch.float64))

In [0]:
#main iterations
max_iter=25
#only affine steps now

# for i in range(max_iter):
